In [1]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

credit_rating_cpd = TabularCPD(
                variable='CR',
                variable_card=4,
                values=[[0.85, 0.04, 0.12, 0.02, 0.13, 0.01],
                        [0.1, 0.07, 0.65, 0.07, 0.2, 0.04],
                        [0.04, 0.75, 0.15, 0.25, 0.45, 0.25],
                        [0.01, 0.14, 0.08, 0.66, 0.22, 0.7]],
                evidence=['OL', 'PH'],
                evidence_card=[3, 2])
credit_rating_cpd = TabularCPD(variable='CR', variable_card=4,
                values=[[0.85, 0.04, 0.12, 0.02, 0.13, 0.01],
                        [0.1, 0.07, 0.65, 0.07, 0.2, 0.04],
                        [0.04, 0.75, 0.15, 0.25, 0.45, 0.25],
                        [0.01, 0.14, 0.08, 0.66, 0.22, 0.7]],
                evidence=['OL', 'PH'], evidence_card=[3, 2])

interest_rate_cpd = TabularCPD(variable='IR', variable_card=3,
                values=[[0.01, 0.05, 0.12, 0.02, 0.05, 0.15, 0.3, 0.4, 0.55, 0.57, 0.83, 0.94],
                        [0.09, 0.7, 0.7, 0.23, 0.4, 0.45, 0.6, 0.55, 0.4, 0.4, 0.15, 0.05],
                        [0.9, 0.25, 0.18, 0.75, 0.55, 0.4, 0.1, 0.05, 0.05, 0.03, 0.02, 0.01]],
                evidence=['CR', 'IL'], evidence_card=[4, 3])

Outstanding_loan_cpd = TabularCPD(variable='OL', variable_card=3, values=[[0.15, 0.55, 0.3]])

Payment_history_cpd = TabularCPD(variable='PH', variable_card=2, values=[[0.8, 0.2]])

Income_level_cpd = TabularCPD(variable='IL', variable_card=3, values=[[0.1, 0.6, 0.3]])

In [10]:
credit_approval_model = BayesianModel([('OL', 'CR'),
                             ('PH', 'CR'),
                             ('IL', 'IR'),
                             ('CR', 'IR')])

In [3]:
credit_approval_model.add_cpds(credit_rating_cpd, interest_rate_cpd, Outstanding_loan_cpd, Payment_history_cpd, Income_level_cpd)
credit_approval_model.check_model()

True

In [6]:
from pgmpy.inference.base import Inference
from pgmpy.factors import factor_product

import itertools


class SimpleInference(Inference):
    def query(self, var, evidence):
        # self.factors is a dict of the form of {node: [factors_involving_node]}
        factors_list = set(itertools.chain(*self.factors.values()))
        product = factor_product(*factors_list)
        reduced_prod = product.reduce(evidence, inplace=False)
        reduced_prod.normalize()
        var_to_marg = set(self.model.nodes()) - set(var) - set([state[0] for state in evidence])
        marg_prod = reduced_prod.marginalize(var_to_marg, inplace=False)
        return marg_prod
    
infer = SimpleInference(credit_approval_model)
ir = infer.query(var=['IR'], evidence=[('OL', 0)])
print(ir)

╒══════╤═══════════╕
│ IR   │   phi(IR) │
╞══════╪═══════════╡
│ IR_0 │    0.1626 │
├──────┼───────────┤
│ IR_1 │    0.5751 │
├──────┼───────────┤
│ IR_2 │    0.2623 │
╘══════╧═══════════╛


In [7]:
infer = SimpleInference(credit_approval_model)
ir = infer.query(var=['IR'], evidence=[('OL', 0),('CR', 0)])
print(ir)

╒══════╤═══════════╕
│ IR   │   phi(IR) │
╞══════╪═══════════╡
│ IR_0 │    0.0670 │
├──────┼───────────┤
│ IR_1 │    0.6390 │
├──────┼───────────┤
│ IR_2 │    0.2940 │
╘══════╧═══════════╛


In [8]:
infer = SimpleInference(credit_approval_model)
ir = infer.query(var=['IR'], evidence=[('OL', 2), ('CR', 0)])
print(ir)

╒══════╤═══════════╕
│ IR   │   phi(IR) │
╞══════╪═══════════╡
│ IR_0 │    0.0670 │
├──────┼───────────┤
│ IR_1 │    0.6390 │
├──────┼───────────┤
│ IR_2 │    0.2940 │
╘══════╧═══════════╛
